In [1]:
import os
os.chdir('../movies')
from movieLens import MovieLens

# Load the movie Lens class
ml = MovieLens()

# Algorithm

In [12]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
# from RBM import RBM

from surprise import accuracy
# import heapq
# from collections import defaultdict
# from operator import itemgetter
import numpy as np
import pandas as pd

In [3]:
ratings = ml.ratings
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
# Method from the Surprise library to load the DataFrame 
# Define the Reader object to parse the dataframe
reader = Reader(rating_scale=(ratings['rating'].min(), ratings['rating'].max()))

# Load the dataframe as a ratings dataset
ratingsDataset = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Build the full trainset
#trainSet = ratingsDataset.build_full_trainset()
trainSet, testSet = train_test_split(ratingsDataset, test_size=0.2, random_state=42)

# PRUEBA 1

Para preparar los datos de train y test, necesitamos crear conjuntos en formato de matriz con cada fila representando a un usuario y cada celda de la fila representando la valoración de cada película, dado que esta es la entrada esperada por el algoritmo RBM.

Para ello, necesitamos el número total de usuarios como números de fila y el número total de películas como número de columna
* num_users = trainSet.n_users
* num_movies = trainSet.n_items

In [5]:
def fitData(trainset):
    #AlgoBase.fit(self, trainset)

    num_users = trainset.n_users
    num_movies = trainset.n_items
    
    # 3D matrix: users, movies and ratings
    # Ratings has size 10 given the possible rating values
    trainingMatrix = np.zeros([num_users, num_movies, 10], dtype=np.float32)

    for (uid, iid, rating) in trainset.all_ratings():
        adjustedRating = int(float(rating)*2.0) - 1
        trainingMatrix[int(uid), int(iid), adjustedRating] = 1

    # Flatten to a 2D array, with nodes for each possible rating type on each possible item, for every user.
    trainingMatrix = np.reshape(trainingMatrix, [trainingMatrix.shape[0], -1])
    
    return trainingMatrix

In [6]:
trainingMatrix = fitData(trainSet)
trainingMatrix.shape

(610, 89280)

In [7]:
from RBM import RBM

# Create an RBM with (num items * rating values) visible nodes
model = RBM(trainingMatrix.shape[1])
model.train(trainingMatrix)

Trained epoch  0
Trained epoch  1
Trained epoch  2
Trained epoch  3
Trained epoch  4
Trained epoch  5
Trained epoch  6
Trained epoch  7
Trained epoch  8
Trained epoch  9
Trained epoch  10
Trained epoch  11
Trained epoch  12
Trained epoch  13
Trained epoch  14
Trained epoch  15
Trained epoch  16
Trained epoch  17
Trained epoch  18
Trained epoch  19


In [8]:
def getPredictions(rbm,num_users,num_movies,trainingMatrix):
    
    predictedRatings = np.zeros([num_users, num_movies], dtype=np.float32)
    
    for uiid in range(num_users):
        if (uiid % 50 == 0):
            print("Processing user ", uiid)
        recs = rbm.getRecommendations([trainingMatrix[uiid]])
        recs = np.reshape(recs, [num_movies, 10])

        for itemID, rec in enumerate(recs):
            # The obvious thing would be to just take the rating with the highest score:                
            #rating = rec.argmax()
            # ... but this just leads to a huge multi-way tie for 5-star predictions.
            # The paper suggests performing normalization over K values to get probabilities
            # and take the expectation as your prediction, so we'll do that instead:

            normalized = np.exp(rec)/np.sum(np.exp(rec), axis=0)
            rating = np.average(np.arange(10), weights=normalized)
            
            predictedRatings[uiid, itemID] = (rating + 1) * 0.5

    return predictedRatings

In [10]:
predictions = getPredictions(model,trainSet.n_users,trainSet.n_items,trainingMatrix)

Processing user  0
Processing user  50
Processing user  100
Processing user  150
Processing user  200
Processing user  250
Processing user  300
Processing user  350
Processing user  400
Processing user  450
Processing user  500
Processing user  550
Processing user  600


In [14]:
predictions

array([[2.779465 , 3.0504014, 2.8211207, ..., 2.765604 , 2.7764707,
        2.758736 ],
       [2.7794168, 3.0504014, 2.8211167, ..., 2.76558  , 2.7764108,
        2.7587187],
       [2.7793686, 3.0497763, 2.8210335, ..., 2.7655556, 2.7763507,
        2.7587013],
       ...,
       [2.7771592, 3.0582588, 2.8242025, ..., 2.7666385, 2.7776616,
        2.7591581],
       [2.7727664, 3.0462308, 2.812768 , ..., 2.7634869, 2.7739844,
        2.7580013],
       [2.7772813, 3.0585678, 2.8244917, ..., 2.7667131, 2.777739 ,
        2.7591944]], dtype=float32)

In [31]:
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

# Prueba 2

In [ ]:
from RBM_v2 import RBM_v2

# Create an RBM with (num items * rating values) visible nodes
model2 = RBM_v2(trainingMatrix.shape[1])
model2.train(trainingMatrix)

In [18]:
predictions = getPredictions(model2,trainSet.n_users,trainSet.n_items,trainingMatrix)

Processing user  0
Processing user  50
Processing user  100
Processing user  150
Processing user  200
Processing user  250
Processing user  300
Processing user  350
Processing user  400
Processing user  450
Processing user  500
Processing user  550
Processing user  600


In [20]:
predictions

array([[2.777783 , 3.0561798, 2.820074 , ..., 2.7891521, 2.7764978,
        2.7632673],
       [2.7777126, 3.0561798, 2.8200672, ..., 2.7890635, 2.7764454,
        2.7632442],
       [2.777642 , 3.0555592, 2.8199842, ..., 2.7889745, 2.7763927,
        2.7632208],
       ...,
       [2.775049 , 3.063708 , 2.8231537, ..., 2.7885175, 2.7779596,
        2.7635498],
       [2.7732291, 3.0508475, 2.8116956, ..., 2.7841167, 2.7738535,
        2.7615008],
       [2.7750912, 3.0640235, 2.823429 , ..., 2.7886112, 2.7780674,
        2.763603 ]], dtype=float32)

In [35]:
model2.test(testSet)

AttributeError: 'RBM_v2' object has no attribute 'trainset'

In [36]:
from surprise.model_selection import train_test_split

# Load the dataframe as a ratings dataset
ratingsDataset = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Split the dataset into train and test sets
trainSet, testSet = train_test_split(ratingsDataset, test_size=0.2, random_state=42)

In [37]:
model2.test(testSet)

AttributeError: 'RBM_v2' object has no attribute 'trainset'

In [29]:
from surprise import accuracy
from surprise import Prediction

# Assuming `testSet` is the list of tuples

# Convert the testSet tuples into Prediction objects
prediction_objects = [Prediction(uid=t[0], iid=t[1], r_ui=t[2], est=predictions[i], details={})
                      for i, t in enumerate(testSet)]

# Calculate RMSE and MAE
rmse = accuracy.rmse(prediction_objects)
mae = accuracy.mae(prediction_objects)


IndexError: index 610 is out of bounds for axis 0 with size 610

# Probar si con RBM AlgotBase se pueden sacar las métricas

In [33]:
from RBMAlgorithm import RBMAlgorithm
RBM = RBMAlgorithm(epochs=20)

In [34]:
RBM.test(testSet)

AttributeError: 'RBMAlgorithm' object has no attribute 'trainset'

# Metrics

In [ ]:
import os
os.chdir('../metrics')
from metrics import evaluationMetrics
em = evaluationMetrics()

In [ ]:
# Get test and antitest predictions
predtest = model.test(testSet)
predantitest = model.test(antitest)

# Get top N recommended movies for each user based on estimated ratings
top_10_SVD = em.getTopN(predantitest,minimumRating = 3.5)

## Métricas de precisión: RMSE y MAE

In [15]:
# # RMSE
rmse = accuracy.rmse(predtest)

# MAE
mae = accuracy.mae(predtest)

# rmse = accuracy.rmse(predictions)
# mae = accuracy.mae(predictions)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

## Métricas de relevancia: Precision, Recall y NDCG

In [ ]:
# Precision
precisions = em.getPrecision(predtest, k=10, threshold=3.5)

# Mean Average Precision
mapModel = np.mean(list(precisions.values()))

# Recall
recalls = em.getRecall(predtest, k=10, threshold=3.5)

# Mean Average Recall
marModel = np.mean(list(recalls.values()))

# Normalized discounted cummulative gain (NDCG)
ndcgs, mean_ndcg = em.getNDCG(predtest,10)

## Otras métricas de interés: Coverage, User Coverage y Novelty

In [ ]:
# Coverage
coverage = em.getCoverage(top_10_SVD,trainSet.n_items,trainSet.all_users())

# User coverage
user_coverage = em.getUserCoverage(top_10_SVD, trainSet.n_users,4)

# Novelty
novelty = em.getNovelty(top_10_SVD,trainSet)

Por último creamos un dataframe con todas las métricas de evaluación asociadas al modelo

In [ ]:
cols = ["Model","RMSE","MAE","MAP","MAR","Mean_NDCG","Coverage","User_Coverage","Novelty"]
metrics_data = []

# Append the results to the list of dictionaries
metrics_data.append({"Model": "item-based", "RMSE": rmse, "MAE": mae, "MAP": mapModel, "MAR": marModel,
                     "Mean_NDCG": mean_ndcg, "Coverage": coverage, "User_Coverage": user_coverage,
                     "Novelty": novelty})

# Convert the list of dictionaries into a DataFrame
metrics_df = pd.DataFrame(metrics_data, columns=cols)
metrics_df

Vamos a evaluar también un modelo Random, en concreto "NormalPredictor" para poder comparar sus resultados con el resto de modelos. NormalPredictor es un algoritmo simple en Surprise que predice calificaciones aleatoriamente basado en la distribución del conjunto de entrenamiento. Supone una distribución normal de las calificaciones y genera predicciones aleatorias en función de esa distribución.

In [ ]:
from surprise import NormalPredictor

Random = NormalPredictor()